In [ ]:
import pandas as pd
import json
import datetime
import time
import os
import numpy as np
from config import DATA_METADATA_PROCESSED_FILE, DATA_BIM_RAW_FOLDER, DATA_ISO_CONSUMPTION_PROCESSED_FILE
import matplotlib

In [ ]:
meatadata =  pd.read_excel(DATA_METADATA_PROCESSED_FILE, sheets='SENSORS')
output_path = DATA_ISO_CONSUMPTION_PROCESSED_FILE
meatadata

In [ ]:
buildings = list(set(meatadata['ID_CEA'].values))
scenarios = {'2018': pd.date_range(start='2018-01-01', end='2018-12-31 23:00:00', freq='H'),
             '2019': pd.date_range(start='2019-01-01', end='2019-12-31 23:00:00', freq='H')}
df = pd.DataFrame()
for building in buildings:
    for scenario, timestamp in scenarios.items():
        sg_path = os.path.join(DATA_BIM_RAW_FOLDER, scenario, 'outputs\data\demand', building+ ".csv")
        sg_gfa_path = os.path.join(DATA_BIM_RAW_FOLDER, scenario, 'outputs\data\demand', "Total_demand.csv")
        total_deamnd = pd.read_csv(sg_gfa_path)
        GFA_data = total_deamnd.loc[total_deamnd['Name']==building, 'GFA_m2'].values[0]
        solar_gains_df = pd.read_csv(sg_path)[['GRID_kWh', 'Name']]
        solar_gains_df['ISO_Whm2'] = solar_gains_df['GRID_kWh']*1000/GFA_data
        solar_gains_df = solar_gains_df.set_index(timestamp)
        solar_gains_df = solar_gains_df.resample('D').sum()
        solar_gains_df['ID_CEA'] = building
        solar_gains_df['timestamp'] = solar_gains_df.index
        df = df.append(solar_gains_df, ignore_index=True)
df
#get information about the sola radiation of buildings.

In [ ]:
df.to_csv(output_path, index=False)

In [ ]:
df = df.set_index(df['timestamp'])
visual = df[df['ID_CEA'] == buildings[24]]
ax = visual['ISO_Whm2'].plot(color = ['C2'])
font = {'family' : 'Arial',
        'size'   : 18}
ax.set_xlabel("")
ax.set_ylabel("Daily Energy Consumption [Wh/m2]")
ax.legend(["ISO Energy Consumption"]);
matplotlib.rc('font', **font)

In [ ]:
df = df.set_index(df['timestamp'])
visual = df[df['ID_CEA'] == buildings[24]]
visual['consumption'] = visual['ISO_Whm2'] * 100 /1000# typical size
ax = visual['consumption'].plot(legend=False)
font = {'family' : 'Arial',
        'size'   : 18}
ax.set_xlabel("")
ax.set_ylabel("Daily Consumption [kWh]")
matplotlib.rc('font', **font)

In [ ]:
from scipy.signal import savgol_filter as sg
p = 2
for si in df['ID_CEA'].unique():
    index = df['ID_CEA'] == si
    df.loc[index, 'ISO_kWh_MEAN_roll_3'] = sg(df[index].ISO_Whm2, 411, p)
df[['ISO_Whm2']][:200].plot()

In [ ]:
df[['ISO_kWh_MEAN_roll_3','ISO_Whm2']][:500].plot()